In [ ]:
import os

os.chdir("drive/My Drive/NLP_Project/adv_oshumed")

In [ ]:
!ls

ohsumed_test_hr.tsv	    ohsumed_test_result.tsv   ohsumed_train.tsv
ohsumed_test_obstruct.tsv   ohsumed_test_suggest.tsv  Untitled0.ipynb
ohsumed_test_physician.tsv  ohsumed_test.tsv


In [ ]:

!ls "drive/My Drive/NLP_Project/adv_oshumed"

ls: cannot access 'drive/My Drive/NLP_Project/adv_oshumed': No such file or directory


In [ ]:
import nltk
nltk.download('stopwords')
!pip install imblearn

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
%matplotlib inline

import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline
import seaborn as sns
from imblearn.over_sampling import SMOTE

# Oshumed Dataset Load

###training dataset

In [ ]:
df_train1 = pd.read_csv('ohsumed_train.tsv')
df_train1['lebel']=df_train1['lebel'].astype(str)

In [ ]:
df_train1

,Unnamed: 0,body,lebel
0,0,Differential roles of opioid receptors in resp...,8
1,1,Control of total peripheral resistance during ...,23
2,2,Contact dermatoses from disposable glove use: ...,17
3,3,Early magnetic resonance imaging in acute trau...,23
4,4,Biguanides and sulfonylureas as combination th...,18
...,...,...,...
3352,3352,Alterations in splanchnic blood flow following...,21
3353,3353,Cup containment and orientation in cemented to...,23
3354,3354,Improvement in early diastolic filling dynamic...,14
3355,3355,The Marinesco-Sjogren syndrome examined by com...,23


one hot encodding

In [ ]:
df_train2=pd.get_dummies(df_train1.lebel)
df_train2

,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3352,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3353,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3354,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3355,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
df_train=pd.concat([df_train1,df_train2], axis=1)

craeting datset containg text and associated one hot encodded class

In [ ]:
df_train.drop(['Unnamed: 0','lebel'],axis=1,inplace=True)
df_train

,body,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,3,4,5,6,7,8,9
0,Differential roles of opioid receptors in resp...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Control of total peripheral resistance during ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,Contact dermatoses from disposable glove use: ...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Early magnetic resonance imaging in acute trau...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,Biguanides and sulfonylureas as combination th...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3352,Alterations in splanchnic blood flow following...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3353,Cup containment and orientation in cemented to...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3354,Improvement in early diastolic filling dynamic...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3355,The Marinesco-Sjogren syndrome examined by com...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


### testing dataset

In [ ]:

df_test1 = pd.read_csv('ohsumed_test.tsv')
df_test1['lebel'] =df_test1['lebel'].astype(str)

In [ ]:
df_test2=pd.get_dummies(df_test1.lebel)
df_test2

,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4038,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4039,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4040,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4041,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_test=pd.concat([df_test1,df_test2], axis=1)

In [ ]:
df_test.drop(['Unnamed: 0','lebel'],axis=1,inplace=True)
df_test

,body,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,3,4,5,6,7,8,9
0,Vascular responsiveness in obstructed gut.\n M...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Type VII collagen and 19-DEJ-1 antigen. Compar...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The cellular biology of bone tumors.\n New kno...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Literacy and laryngectomy: how should one trea...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,The painful shoulder: Part I. Extrinsic disord...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4038,Hypoxia induces a specific set of stress prote...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4039,Role of 5 alpha-reductase inhibitors in the tr...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4040,Staphylococcus aureus infection of human endot...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4041,Dietary alpha-linolenic acid and immunocompete...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


### tmp_testing dataset

In [ ]:

df_test11 = pd.read_csv('ohsumed_test_yr_wk_hr.tsv',sep='\t')
df_test11['lebel'] =df_test11['lebel'].astype(str)

In [ ]:
df_test21=pd.get_dummies(df_test11.lebel)
df_test21

,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4038,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4039,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4040,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4041,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_test_hr=pd.concat([df_test11,df_test21], axis=1)

In [ ]:
df_test_hr.drop(['Unnamed: 0','lebel'],axis=1,inplace=True)
df_test_hr

,body,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,3,4,5,6,7,8,9
0,Vascular responsiveness in obstructed gut. Mu...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Type VII collagen and 19-DEJ-1 antigen. Compar...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The cellular biology of bone tumors. New know...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Literacy and laryngectomy: how should one trea...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,The painful shoulder: Part I. Extrinsic disord...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4038,Hypoxia induces a specific set of stress prote...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4039,Role of 5 alpha-reductase inhibitors in the tr...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4040,Staphylococcus aureus infection of human endot...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4041,Dietary alpha-linolenic acid and immunocompete...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


#Data cleaning and processing

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [ ]:
categories = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23']

In [ ]:
df_train['body'] = df_train['body'].map(lambda com : clean_text(com))
df_test['body'] = df_test['body'].map(lambda com : clean_text(com))
df_test_hr['body'] = df_test_hr['body'].map(lambda com : clean_text(com))
df_train

,body,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,3,4,5,6,7,8,9
0,differential roles of opioid receptors in resp...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,control of total peripheral resistance during ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,contact dermatoses from disposable glove use a...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,early magnetic resonance imaging in acute trau...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,biguanides and sulfonylureas as combination th...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3352,alterations in splanchnic blood flow following...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3353,cup containment and orientation in cemented to...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3354,improvement in early diastolic filling dynamic...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3355,the marinesco sjogren syndrome examined by com...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
df_test_hr['body'] = df_test_hr['body'].map(lambda com : clean_text(com))

In [ ]:
X_train = df_train.body
X_test = df_test.body
X_test_hr=df_test_hr.body
print(X_train.shape)
print(X_test.shape)
print(X_test_hr.shape)

(3357,)
(4043,)
(4043,)


# LogReg Model with percentage(%) of flips

In [ ]:
smt = SMOTE(random_state=42)

In [ ]:
from sklearn.metrics import confusion_matrix
LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('smt',smt),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    LogReg_pipeline.fit(X_train, df_train[category])
    # compute the testing accuracy
    prediction = LogReg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(df_test[category], prediction)))
    print(classification_report(df_test[category],prediction))
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    LogReg_pipeline.fit(X_train, df_train[category])
    # compute the testing accuracy
    prediction_hr = LogReg_pipeline.predict(X_test_hr)
    print('Test accuracy is {}'.format(accuracy_score(df_test_hr[category], prediction_hr)))
    print(classification_report(df_test_hr[category],prediction_hr))
    a=0
    for i in range(4043):
      if prediction[i]!=prediction_hr[i]:
        a=a+1
    b=(a*100)/4043
    print('total number of flip is',a)
    print('Percentage of flips is',b)

... Processing 1


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Test accuracy is 0.9821914419985159
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3941
           1       0.71      0.49      0.58       102

    accuracy                           0.98      4043
   macro avg       0.85      0.74      0.79      4043
weighted avg       0.98      0.98      0.98      4043

... Processing 1


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Test accuracy is 0.9821914419985159
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3941
           1       0.71      0.49      0.58       102

    accuracy                           0.98      4043
   macro avg       0.85      0.74      0.79      4043
weighted avg       0.98      0.98      0.98      4043

total number of flip is 0
Percentage of flips is 0.0
... Processing 2


...

Test accuracy is 0.9975265891664605
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4033
           1       0.00      0.00      0.00        10

    accuracy                           1.00      4043
   macro avg       0.50      0.50      0.50      4043
weighted avg       1.00      1.00      1.00      4043

total number of flip is 0
Percentage of flips is 0.0
... Processing 23


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Test accuracy is 0.8951273806579273
              precision    recall  f1-score   support

           0       0.93      0.96      0.94      3624
           1       0.49      0.33      0.39       419

    accuracy                           0.90      4043
   macro avg       0.71      0.64      0.67      4043
weighted avg       0.88      0.90      0.89      4043

... Processing 23


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Test accuracy is 0.8958694039079891
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      3624
           1       0.50      0.32      0.39       419

    accuracy                           0.90      4043
   macro avg       0.71      0.64      0.67      4043
weighted avg       0.88      0.90      0.89      4043

total number of flip is 7
Percentage of flips is 0.17313875834776157
